In [70]:
import pandas as pd
from dateutil import parser
import datetime

In [71]:
DIR = "/Users/damoncrockett/astrodata/tables/"

In [72]:
df = pd.read_csv(DIR+"astrodata_moon_galaxy.csv")

In [73]:
df.date_taken = df.date_taken.apply(parser.parse)

In [74]:
df['cdate'] = [item.date() for item in df.date_taken]

In [75]:
t0 = df.cdate.min()

In [76]:
df['secpast'] = [(item-t0).total_seconds() for item in df.cdate]

In [77]:
from PIL import Image,ImageDraw

In [78]:
nxbin = 234

In [79]:
df['tbin'] = pd.cut(df.secpast,nxbin,labels=False)

In [80]:
uph = df.tbin[df.galaxy==True].value_counts().max()
downh = df.tbin[df.moon==True].value_counts().max()

In [81]:
width = nxbin
height = uph + downh

In [82]:
side = 64

In [83]:
px_w = width * side
px_h = height * side

In [84]:
canvas = Image.new('RGB',(px_w,px_h),"black")

In [85]:
draw = ImageDraw.Draw(canvas)

In [86]:
for binn in df.tbin.unique():
    xpos = binn * side
    
    for subj in ['moon','galaxy']:
        if subj=='galaxy':
            tmp = df[df.tbin==binn]
            tmp = tmp[tmp.galaxy==True]
            tmp.reset_index(drop=True,inplace=True)
            ypos = uph * side - side # one img height above the equator

            for i in tmp.index:
                local_path = tmp.local_path.loc[i]
                im = Image.open(local_path)
                im.thumbnail((side,side),Image.ANTIALIAS)
                canvas.paste(im,(xpos,ypos))
                ypos = ypos - side
        
        elif subj=='moon':
            tmp = df[df.tbin==binn]
            tmp = tmp[tmp.moon==True]
            tmp.reset_index(drop=True,inplace=True)
            ypos = uph * side # top left corner img at equator

            for i in tmp.index:
                local_path = tmp.local_path.loc[i]
                im = Image.open(local_path)
                im.thumbnail((side,side),Image.ANTIALIAS)
                canvas.paste(im,(xpos,ypos))
                ypos = ypos + side

In [87]:
draw.line([(0,uph*side),(px_w,uph*side)],fill='white',width=3)

In [88]:
canvas.save("/Users/damoncrockett/Desktop/moongalhist.png")